# HuggingFace Transformers
## 02 Advanced tutorial
제작 : 이유경 (yukyung_lee@korea.ac.kr)

In [1]:
! pip install transformers -q

     |████████████████████████████████| 2.1 MB 9.5 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
     |████████████████████████████████| 895 kB 57.3 MB/s 


## 💻 Advanced tutorial
* 아래와 같은 내용을 정리합니다
    * 01 : Token 추가하기
    * 02 : [CLS] output 추출하기
        * 참고 : [CLS] 토큰은 정말 문장을 대표할까 ?
        

### 01 Token 추가하기

간혹 모델의 성능을 높이기 위해 special token을 추가하거나, domain에 특화된 단어를 추가해주는 방법이 있습니다. 

* special token을 추가하는 경우 해당 token이 special token임을 tokenizer에게 알려주어야 합니다.

  : 따라서 이 경우에는 `add_special_tokens()` 메서드를 사용해야합니다.

* 일반 token을 추가하는 경우엔 `add_tokens()` 메서드를 사용하여 vocab을 늘려줄 수 있습니다.

* tokenizer에 vocab을 추가했다면 pretrained model의 token embedding 사이즈를 변경해주어야합니다.

  : `model.resize_token_embedding`을 이용하면 됩니다.
  
  : tokenizer는 `len()` 사용하면 vocab의 총 개수가 나오므로 이를 이용하면 됩니다

  : 추가한 개수 만큼 vocab을 늘려주고, embedding 사이즈도 늘려주는 과정을 통해 직관적으로 vocab을 추가합니다
 



* Reference code

  [special token 추가하기](https://github.com/huggingface/tokenizers/issues/247#issuecomment-675458087)

  [token 추가하기](https://medium.com/@pierre_guillou/nlp-how-to-add-a-domain-specific-vocabulary-new-tokens-to-a-subword-tokenizer-already-trained-33ab15613a41)


In [3]:
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForQuestionAnswering

In [4]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

# special token 추가하기 
special_tokens_dict = {'additional_special_tokens': ['[special1]','[special2]','[special3]','[special4]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# token 추가하기
new_tokens = ['COVID', 'hospitalization']
num_added_toks = tokenizer.add_tokens(new_tokens)

# 기존 config로 모델을 불러오기
# 모델을 불러오기전에 vocab을 수정하면 pretrained config와 충돌이 일어나 에러가 발생하니 주의
model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
)

# tokenizer config 수정해주기 (추후에 발생할 에러를 줄이기 위해)
config.vocab_size = len(tokenizer)

# model의 token embedding 사이즈 수정하기
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model chec

Embedding(32006, 768)

**Q : special token을 추가할 때 항상 resize를 해주어야 하나요 ?**

A : 꼭 그렇지 않습니다. 잘 만들어진 모델은 resize를 하지않고도 모델에 새로운 vocab을 추가할 수 있도록 여분의 vocab 자리를 만들어 두었습니다. *여분의 vocab 개수는 모델에 따라 다르니 확인이 필요합니다.*

아래의 코드를 살펴봅시다

In [5]:
model_name = 'klue/bert-base'

config = AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

In [6]:
tokenizer.vocab["[unused0]"]

31500

In [7]:
# klue/bert-base는 500개(index:0~499) dummy vocab을 가지고 있음
tokenizer.vocab["[unused499]"]

31999

`tokenizer.vocab`에 특정 단어가 포함되어있을지 확인해보려면 단어를 string 타입으로 넣어서 확인해보면 됩니다. 

`[unused0]`이라는 인풋이 보시이나요 ? 사용하지 않는 dummy vocab을 추가했다는 의미입니다. 

* **이러한 dummy vocab을 추가하는 이유는 무엇일까요 ?**
  : 사용자의 니즈에 따라서 단어를 추가할 수 있는 여유 공간을 제공한 것입니다. 즉, 유저가 pretrained model을 최대한 수정하지 않고도 다양한 vocab을 사용할 수 있도록 의도한 것입니다.

* 최근에 공개된 모델들은 대부분 'unused' vocab을 가지고 있습니다.

* 즉, **모든 모델이 dummy vocab을 고려하는것은 아닙니다.**
  * 예를들어, SKT의 KoBERT는 dummy vocab을 가지고 있지 않습니다.
  * 따라서 추가 vocab을 넣을 경우에는 manual 하게 수정을 해줘야 하며, gluonnlp를 사용하는 부분을 수정해야합니다. 
  * model에 vocab을 추가하는것은 resize method를 사용하면 되지만, tokenizer부분을 바꿔주기위해서는 tokenizer의 vocab을 수정해야합니다.
  * 다만 transformers로 converting된 KoBERT를 사용하는 경우에는 transformers의 메서드를 적용할 수 있습니다.


In [8]:
model_name = 'bert-base-cased'

config = AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)

In [9]:
# bert-base-cased 모델은 100개(index :1~101)의 dummy vocab을 가지고 있음
tokenizer.vocab["[unused101]"]

105

모델별로 dummy vocab을 위한 자리가 미리 마련된걸 알았으니, tokenizer loading시 cache가 저장되는 디렉토리로 이동해서 vocab.txt 파일을 manual하게 변경해주면 resize 없이 사용할 수 있습니다. (귀찮다면 add_token 후 resize를 합시다)

* vocab을 매우 많이 추가했다면 pretraining을 다시 수행하는것이 좋습니다 (TAPT: Task Adaptive PreTraining)

  * TAPT는 아래의 논문을 참고하세요 🙂

    : [Don't Stop Pretraining: Adapt Language Models to Domains and Tasks](https://arxiv.org/abs/2004.10964)

* 그렇지 않다면 finetuning만 수행해도 충분합니다

### 02 [CLS] output 추출하기

model에서 [CLS] 자리의 embedding만 가지고 오고 싶은 경우가 있습니다. 

이때 전체 output representation에서 indexing으로 `[CLS]`embedding을 가지고 올 수 도 있지만 

`.pooler_output` 을 이용하면 보다 쉽게 값을 가져올 수 있습니다


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

inputs = tokenizer("NLP는 정말 재미있어", return_tensors="pt")
outputs = model(**inputs)

cls_output = outputs.pooler_output

#### 📣 참고 : [CLS] 토큰은 정말 문장을 대표할까 ?

> 해당 파트는 참고용입니다 :) 관심이 있으시다면 읽어보셔도 좋습니다.

BERT paper를 살펴보면 [CLS] 토큰은 문장을 대표하는 값으로 알려져 있습니다. 

**의심의 여지 없이 문장을 대표할까요 ?**

놀랍게도, BERT의 저자 또한 [CLS]가 Sentence representation이란걸 보장할 순 없다고 밝혔습니다.

아래의 이슈를 보시면 매우 흥미로운 주제로 이야기를 나누고 있습니다
https://github.com/google-research/bert/issues/164

* 우리가 특정 task를 수행할 때 `[CLS]` 토큰이 '당연히' 문장을 대표해줄것이라는 가정을 가지는것은 위험합니다.

* 실험을 통해 여러분이 수행하고자 하는 task에 어떤 값이 중요할지 확인해보세요 ! 

* **읽어볼만한 논문 추천 (SBERT)**
  
  : 논문 : https://arxiv.org/pdf/1908.10084.pdf

  : ***The most commonly used approach is to average the BERT output layer (known as BERT embeddings) or by using the output of the first token (the [CLS] token). As we will show, this common practice yields rather bad sentence embeddings, often worse than averaging GloVe embeddings (Pennington et al., 2014).***

  : 요약) avg나 CLS를 사용하는게 일반적이지만 이건 Glove
  embedding의 avg보다도 성능이 낮다.

  : 저는 해당 논문을 읽고 `[CLS]`가 sentence 를 대표하지 못하겠구나를 처음 인지하게 되었습니다. 

  : input에 대한 representation을 추출한 후 pooling layer를 쌓아 maxpooling이나 average pooling을 수행하기도 합니다.



### [TIP!] 🚪✊Knock Knock을 활용하여 모델 학습 완료 알림받기

🚪✊Knock Knock

* 학습 후 메일, 슬랙 등 원하는 곳으로 학습 종료 알람을 해주는 라이브러리입니다

* huggingface에서 만든 공식 라이브러리니 사용해보시는걸 추천드립니다

* 더 자세한 내용은 아래 링크를 참고하세요 !
https://github.com/huggingface/knockknock

* 아래의 코드를 삽입하면 학습 후 슬랙 알림을 받을 수 있습니다

> pip install knockknock

```
from knockknock import slack_sender

webhook_url = "<webhook_url_to_your_slack_room>"
@slack_sender(webhook_url=webhook_url, channel="<your_favorite_slack_channel>")
def train_your_nicest_model(your_nicest_parameters):
    import time
    time.sleep(10000)
    return {'loss': 0.9} # Optional return value

```


command line에서 실행하기
```
knockknock slack \
    --webhook-url <webhook_url_to_your_slack_room> \
    --channel <your_favorite_slack_channel> \
    sleep 10

```

